In [1]:
import pandas as pd

Prepare the economical data

In [5]:
raw_econ_df = pd.read_csv("data\Annual_FY_June2024.csv")

303


In [9]:
# raw_econ_df.interpolate(method='quadratic')

In [12]:
econ_df = raw_econ_df.ffill()
econ_df = raw_econ_df.bfill()
econ_df.isna().sum().sum()

np.int64(0)

Prepare the interest rates

In [21]:
raw_interest_rates_df = pd.read_csv('data\FEDFUNDS.csv')
raw_interest_rates_df.isna().sum().sum()

np.int64(0)

In [22]:
raw_interest_rates_df['DATE'] = pd.to_datetime(raw_interest_rates_df['DATE'])
raw_interest_rates_df['DATE'] = raw_interest_rates_df['DATE'].dt.year

In [23]:
raw_interest_rates_df

,DATE,FEDFUNDS
0,1955,1.7850000000000000
1,1956,2.7283333333333333
2,1957,3.1050000000000000
3,1958,1.5725000000000000
4,1959,3.3050000000000000
...,...,...
65,2020,0.37583333333333333333
66,2021,0.08000000000000000000
67,2022,1.6833333333333333
68,2023,5.0241666666666667


Prepare the data for the LSTM